### PyTorch Geometric Basics
This section will walk you through the basics of PyG. Essentially, it will cover `torch_geometric.data` and
`torch_geometric.nn`. You will learn how to pass geometric data into your GNN, and how to design a custom MessagePassing
layer, the core of GNN.

#### Data
The torch_geometric.data module contains a Data class that allows you to create graphs from your data very easily.
You only need to specify:
- the attributes/features associated with each node
- the connectivity/adjacency of each node (edge index)

In [1]:
import torch
from torch_geometric.data import Data


x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

edge_index = torch.tensor([[0, 1, 2, 1, 0, 3],
                           [1, 0, 1, 2, 3, 0]], dtype=torch.long)

edge_index_ = torch.tensor([[0, 2, 1, 0, 3],
                           [3, 1, 0, 1, 2]], dtype=torch.long)

edge_attr = torch.tensor([[1,1,0], [1,1,0], [1,1,0], [1,1,0], [1,1,0], [1,1,0]])

data = Data(x=x, y=y, edge_index=edge_index, edge_attr=edge_attr)

In [2]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
# Dataset contains 600 Graphs

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [35]:
data_0 = dataset[0]

print(data_0.is_directed())  # undirected graph
print(data_0.edge_index)     # 168 directed edges
print(data_0.num_edges)

print(data_0.x)              # 37 nodes, each of them has 3 features
print(data_0.num_nodes)

print(data_0.y)              # groundtruth of the graph class


False
tensor([[ 0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
          3,  3,  4,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,
          7,  8,  8,  8,  8,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12,
         12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 16, 16,
         16, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20,
         21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25,
         25, 26, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 29, 29,
         30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 32],
        [ 1,  2,  3,  0,  2,  3, 18, 19,  0,  1,  3,  4, 16, 19,  0,  1,  2,  4,
          5, 16,  2,  3,  5,  6,  3,  4,  6,  7,  4,  5,  7, 20,  5,  6,  8, 20,
         22,  7, 21, 22, 23, 10, 11, 29,  9, 11, 12, 26,  9, 10, 12, 13, 27, 10,
         11, 13, 11, 12, 14, 30, 32, 13, 15, 31, 14, 31, 32,  2,  3, 17, 18, 19,
         20, 16, 18, 19,  1, 16, 17, 19,  1, 

In [48]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in loader:
    print(batch)

Batch(batch=[1035], edge_index=[2, 3984], x=[1035, 21], y=[32])
Batch(batch=[964], edge_index=[2, 3862], x=[964, 21], y=[32])
Batch(batch=[1050], edge_index=[2, 3690], x=[1050, 21], y=[32])
Batch(batch=[1128], edge_index=[2, 4074], x=[1128, 21], y=[32])
Batch(batch=[906], edge_index=[2, 3516], x=[906, 21], y=[32])
Batch(batch=[1153], edge_index=[2, 4038], x=[1153, 21], y=[32])
Batch(batch=[938], edge_index=[2, 3498], x=[938, 21], y=[32])
Batch(batch=[1107], edge_index=[2, 4278], x=[1107, 21], y=[32])
Batch(batch=[947], edge_index=[2, 3696], x=[947, 21], y=[32])
Batch(batch=[967], edge_index=[2, 3766], x=[967, 21], y=[32])
Batch(batch=[1154], edge_index=[2, 4296], x=[1154, 21], y=[32])
Batch(batch=[1084], edge_index=[2, 3996], x=[1084, 21], y=[32])
Batch(batch=[991], edge_index=[2, 3878], x=[991, 21], y=[32])
Batch(batch=[1051], edge_index=[2, 4154], x=[1051, 21], y=[32])
Batch(batch=[1018], edge_index=[2, 3972], x=[1018, 21], y=[32])
Batch(batch=[1047], edge_index=[2, 4038], x=[1047, 2

In [49]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

So there are 4 nodes in the graph, v1 - v4, each of which is associated with a 2-dimensional feature vector, and a label
 `y` indicating its class. These two can be represented as FloatTensors.

The graph connectivity (edge index) should be confined with the COO format, i.e. the first list contains the index of
the source nodes, while the index of target nodes is specified in the second list.

Note that the order of the edge index is irrelevant to the Data object you create since such information is only for
computing the adjacency matrix. Therefore, the above edge_index express the same information as the following one.

We use `torch_geometric.data` to put feature `x`, label `y`, and `edge_index` together.

In our case, the node should be each hand and object. The feature of the node should be bounding box / extracted features
(tbd). The edges between node should be undirected. The features of edge should be distance between node or others (tbd).
The label of each node should be: action id for hand and relative action, idle id for other unused object.

